# 6 month data analysis


In [436]:
import pandas as pd;
from datetime import timedelta;
import plotly.graph_objects as go;


## Importing data from Baby Buddy


Import all .csv data sets exported from the [Baby Buddy](https://github.com/babybuddy/babybuddy) app:


In [437]:
df_sleep = pd.read_csv('data/sleep.csv');
df_feeding = pd.read_csv('data/feeding.csv');
df_diaper = pd.read_csv('data/diaper.csv');
df_weight = pd.read_csv('data/weight.csv');

Transform all date string to Date type:


In [438]:
df_sleep['start'] = pd.to_datetime(df_sleep['start'], format='%d/%m/%Y %H:%M');
df_sleep['end'] = pd.to_datetime(df_sleep['end'], format='%d/%m/%Y %H:%M');
df_feeding['start'] = pd.to_datetime(df_feeding['start'], format='%d/%m/%Y %H:%M');
df_feeding['end'] = pd.to_datetime(df_feeding['end'], format='%d/%m/%Y %H:%M');
df_diaper['time'] = pd.to_datetime(df_diaper['time'], format='%d/%m/%Y %H:%M');
df_weight['date'] = pd.to_datetime(df_weight['date'], format='%d/%m/%Y');

Remove any data points more recent than the 6 month mark:


In [439]:
df_sleep.drop(df_sleep[df_sleep.start >= '2024-02-15'].index, inplace=True);
df_feeding.drop(df_feeding[df_feeding.start >= '2024-02-15'].index, inplace=True);
df_diaper.drop(df_diaper[df_diaper.time >= '2024-02-15'].index, inplace=True);
df_weight.drop(df_weight[df_weight.date >= '2024-02-15'].index, inplace=True);

Add duration columns:


In [440]:
df_sleep['duration'] = df_sleep['end'] - df_sleep['start'];
df_feeding['duration'] = df_feeding['end'] - df_feeding['start'];

## Time Analysis


Total time of 6 months since baby was born is 184 days. Unfortunately we only have sleep data since the 5th of September, which is 163 days:


In [441]:
total_time = pd.to_timedelta(24*7, 'h');
time_asleep = df_sleep[['start','duration']].groupby(pd.Grouper(key='start', freq='1W')).sum().reset_index();
time_asleep['awake'] = total_time - time_asleep['duration'];

time_feeding = df_feeding[['start','duration']].groupby(pd.Grouper(key='start', freq='1W')).sum().reset_index();
time_feeding['partying'] = time_asleep['awake'] - time_feeding['duration'];

In [442]:
time_asleep = time_asleep[1:-1];
time_feeding = time_feeding[1:-1];

In [443]:
def createFrame(week):
    total_hours = 24*7;
    asleep_hours = time_asleep.loc[week]['duration'].total_seconds()//3600;
    awake_hours = time_asleep.loc[week]['awake'].total_seconds()//3600;
    feeding_hours = time_feeding.loc[week]['duration'].total_seconds()//3600;
    partying_hours = time_feeding.loc[week]['partying'].total_seconds()//3600;

    pct_asleep = f"{asleep_hours / total_hours:.0%}";
    pct_awake = f"{awake_hours / total_hours:.0%}";
    pct_feeding = f"{feeding_hours / total_hours:.0%}";
    pct_partying = f"{partying_hours / total_hours:.0%}";


    return {"data": [go.Sankey(
    arrangement='snap',
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label =  [
          f'Total Time', 
          f'Asleep ({pct_asleep})', 
          f'Awake ({pct_awake})', 
          f'Feeding ({pct_feeding})', 
          f'Partying ({pct_partying})'],
      x= [0, 0.5, 0.5, 1, 1],
      y = [0.2, 0.2, 0.5, 0.5, 0.5],
      align = 'left'
    ),
    # Add links
    link = dict(
      source =  [0, 0, 2, 2],
      target =  [1, 2, 3, 4],
      value =  [
          asleep_hours,
          awake_hours, 
          feeding_hours, 
          partying_hours
        ]
))],
"layout": go.Layout(
  title=f'Baby\'s time management (Week {week}) 🕥')}

In [444]:
frames = [];
for week in range(1,len(time_asleep)):
    frames.append(go.Frame(createFrame(week)))

In [445]:
fig = go.Figure(data=createFrame(1)['data'],
layout=go.Layout(
  title=f'Baby\'s time management (Week 1) 🕥',
  updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 600, "redraw": True},
                                "fromcurrent": True, "transition": {"duration": 0,
                                                                    "easing": "quadratic-in-out"}}])])]
),
frames=frames
)
fig.show()

## Diaper analysis


Sum the number of solid diapers per day and then calculate the average per week:


In [446]:
df_poops_per_day = df_diaper[['solid', 'time']].groupby(pd.Grouper(key='time', freq='1D')).sum().reset_index()
df_poops_per_week_avg = df_poops_per_day.groupby(pd.Grouper(key='time', freq='7D')).mean().reset_index()

In [447]:
df_poops_per_week_avg.head()

,time,solid
0,2023-08-15,3.857143
1,2023-08-22,5.714286
2,2023-08-29,2.571429
3,2023-09-05,0.857143
4,2023-09-12,1.428571


Plot the data on a bar chart:


In [448]:
x = df_poops_per_week_avg['time']
y = df_poops_per_week_avg['solid']

fig = go.Figure()
fig.add_trace(go.Bar(
                    y=y,
                    x=x,
              marker_color='#7A5901'
))
fig.update_layout(bargap=0.2, title_text='Average poops per day 💩')
fig.show()